# Sentiment Classification & Embedding II

* Embedding Layer
* Sequence Model

# 01. What data we use?

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt'
data= pd.read_table(url, names=['label', 'reviews'])

In [ ]:
data.head()

,label,reviews
0,0,노래가 너무 적음
1,0,"돌겠네 진짜. 황숙아, 어크 공장 그만 돌려라. 죽는다."
2,1,막노동 체험판 막노동 하는사람인데 장비를 내가 사야돼 뭐지
3,1,차악!차악!!차악!!! 정말 이래서 왕국을 되찾을 수 있는거야??
4,1,시간 때우기에 좋음.. 도전과제는 50시간이면 다 깰 수 있어요


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=2021)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [ ]:
train_data.shape, test_data.shape

((80000, 2), (20000, 2))

In [ ]:
test_data.head()

,label,reviews
0,0,절대로 사지마세요 님들아이거 사서 열심히 설치해서 해봤는데 멀티밖에 안되는 게임이 ...
1,1,조금 해봤는데 재미있네요
2,1,제작자가 행동 하나하나를 다 꿰뚫고있음 뭔 병신짓을 해도 다 받아줘서 좋음 10/10
3,1,길이길이 남을 대작 게임.설명하려 들고 싶지 않을 정도로 완벽 그자체. 꼭 해보세요!
4,0,한글문제인건지 실행은되나 리그 시작 자체가 안됨. 돈받고 파는거면 적어도 어떤 환경...


# Q1. Tokenizing & Text to Sequences

* 상위 40000개 단어만 사용한다.

In [ ]:
## 위의 한국어 전처리를 했다면 사용하지 않는다.
x_train = train_data['reviews'].astype('str').tolist()
x_test = test_data['reviews'].astype('str').tolist()

In [ ]:
y_train = train_data['label'].values
y_test = test_data['label'].values

In [ ]:
### Tokenizer here
from tensorflow.keras.preprocessing.text import Tokenizer
max_words = 40000
tokenizer = Tokenizer(num_words = max_words,lower = False)

In [ ]:
### Tokenizer here
%%time
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)


CPU times: user 4.76 s, sys: 168 ms, total: 4.93 s
Wall time: 4.99 s


In [ ]:
# Text --> Sequence



# Padding Sequence

* 문장의 최대 길이는 40으로 한다.

In [40]:
max_words = max_words
embedding_dim = 128
max_len = 40

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#### Pad Sequences here
x_train=pad_sequences( x_train, maxlen=max_len)
x_test=pad_sequences( x_test, maxlen=max_len)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)

# 0으로 가득찬 데이터에 대한 추가 전처리

In [ ]:
not0train_idx = x_train.sum(1) > 0
not0test_idx = x_test.sum(1) > 0

In [ ]:
train_data = train_data.loc[not0train_idx].reset_index(drop=True)
test_data = test_data.loc[not0test_idx].reset_index(drop=True)

In [ ]:
x_train = x_train[not0train_idx]
y_train = y_train[not0train_idx]

x_test = x_test[not0test_idx]
y_test = y_test[not0test_idx]

In [44]:
x_train.shape

(76916, 40)

# 모델링

**모델 구조**
1. 임베딩 레이어 : 임베딩차원은 128
2. Conv1D 블록 : 필터수 64개, 윈도우 사이즈 5
3. Bidirectional layer :
    * 정방향 : LSTM, 히든스테이트 32 
    * 역방향 : LSTM, 히든스테이트 32
4. Bidirectional layer :
    * 정방향 : GRU, 히든스테이트 32
    * 역방향 : RNN, 히든스테이트 16
5. Conv1D 블록 : 필터수 32개, 윈도우 사이즈 5
6. MaxPool1D 블록 : 필터사이즈2
7. 플래튼
8. FC Layer : 노드 1024개
9. 시그모이드 레이어

In [ ]:
import tensorflow
from tensorflow import keras
from keras.backend import clear_session
from keras.layers import Input, Dense, Flatten, Conv1D, MaxPool1D, LSTM, GRU, RNN, Bidirectional, Embedding
from keras.models import Model


In [50]:
def LSTM_layer(hidden_state, direction) :
    return LSTM(hidden_state, return_sequences=True, go_backwards=direction)
def GRU_layer(hidden_state, direction) :
    return GRU(hidden_state, return_sequences=True, go_backwards=direction)
def RNN_layer(hidden_state, direction) :
    return RNN(hidden_state, return_sequences = True, go_backwards=direction)

In [ ]:
x_train.shape, y_train.shape

((76916, 40), (76916,))

In [52]:
#####################
## your codes here ##
#####################
clear_session()
# 1. 임베딩 레이어 : 임베딩차원은 128
il = Input(shape = (max_len,))
el = Embedding(max_words,
               embedding_dim,
                input_length = max_len)(il)
# 2. Conv1D 블록 : 필터수 64개, 윈도우 사이즈 5
cl1 = Conv1D(filters = 64, 
             kernel_size = 5, 
             activation = 'swish')(el)
# 3. Bidirectional layer :
#     * 정방향 : LSTM, 히든스테이트 32 
#     * 역방향 : LSTM, 히든스테이트 32

bl1 = Bidirectional(LSTM_layer(32, False), backward_layer = LSTM_layer(32, True))(cl1)

# 4. Bidirectional layer :
#     * 정방향 : GRU, 히든스테이트 32
#     * 역방향 : RNN, 히든스테이트 16
bl2 = Bidirectional(GRU_layer(32, False), backward_layer = LSTM_layer(16, True))(bl1)
# 5. Conv1D 블록 : 필터수 32개, 윈도우 사이즈 5
cl2 = Conv1D(filters = 32,
             kernel_size = 5,
             activation = 'swish')(bl2)
# 6. MaxPool1D 블록 : 필터사이즈2
ml = MaxPool1D(2)(cl2)
# 7. 플래튼
fl = Flatten()(ml)
# 8. FC Layer : 노드 1024개
fc_l = Dense(1024, activation = 'swish')(fl)
#9. 시그모이드 레이어
ol = Dense(1, activation = 'sigmoid')(fc_l)

model = Model(il, ol)

model.compile(loss = 'binary_crossentropy',metrics = ['accuracy'], optimizer = 'adam')

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 40)]              0         
                                                                 
 embedding (Embedding)       (None, 40, 128)           5120000   
                                                                 
 conv1d (Conv1D)             (None, 36, 64)            41024     
                                                                 
 bidirectional (Bidirectiona  (None, 36, 64)           24832     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 36, 48)           14592     
 nal)                                                            
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 32)            7712  

In [ ]:
#####################
## your codes here ##
#####################






# EarlyStopping을 이용한 학습.

1. 20%는 벨리데이션 셋.
2. 4epochs전과 비교하여 early stopping할 것.

In [ ]:
#####################
## your codes here ##
#####################




In [ ]:
#####################
## your codes here ##
#####################






In [ ]:
review_idx = 128

temp = test_data.loc[review_idx]
docu = temp['reviews']
label = 'positive' if temp['label'] ==1 else 'Negative'

print(f"문서 번호 {review_idx}")
print(label, " : ", docu)
y_pred = model.predict(x_test[review_idx:review_idx+1])
label_pred = 'positive' if y_pred[0,0] >=0.5 else 'Negative'
print(f"모델의 예측 : {label_pred},   prob = {y_pred[0,0]*100:.2f}%")